In [1]:
import sqlite3
import pandas as pd
import os 

In [2]:
os.listdir('./scraper')

['.idea', 'FIGHTscraper', 'ufcfights.db', 'UFCscraper', 'venv']

In [3]:
#connect to database found in the scraper foler
conn = sqlite3.connect('./scraper/ufcfights.db')
curr = conn.cursor()

In [4]:
#get the names of all the tables found in the ufcfights database
curr.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
fight_tables = curr.fetchall()
print(fight_tables)

[('fighters_tb',), ('fight_tb',), ('fight_stats_tb',)]


In [5]:
#save the sql tables as a pandas df
fighters_df = pd.read_sql_query("""SELECT * FROM %s""" % fight_tables[0][0],conn)
fights_df = pd.read_sql_query("""SELECT * FROM %s""" % fight_tables[1][0],conn)
fight_stats_df = pd.read_sql_query("""SELECT * FROM %s""" % fight_tables[2][0],conn)
#close connection to database
conn.close()

# DATA CLEANING
> HEIGHT to INCHES   
> EXTRACT Numerical Values From Column  
> STRING to DATE  
> EXTRACT WIN LOSE DRAW FROM RECORD  
> HANDLING MISSING VALUES

In [6]:
fighters_df.head()

,name,nick_name,height,weight,reach,stance,date_birth,record,SLpm,StrAcc,SApm,StrDef,TDavg,TDacc,TDdef,SubAvg
0,Mark Eddiva,,"5' 8""",145 lbs.,"68""",Orthodox,"Feb 16, 1986",Record: 6-3-0,3.37,45%,2.98,54%,1.45,22%,63%,0.0
1,Scott Fiedler,,"6' 4""",235 lbs.,--,Orthodox,--,Record: 0-1-0,0.00,0%,0.00,0%,0.00,0%,0%,0.0
2,Drew Fickett,The Master,"5' 10""",155 lbs.,"70""",Orthodox,"Dec 14, 1979",Record: 42-21-0,2.33,41%,2.29,57%,1.30,35%,30%,2.2
3,Marcos Galvao,Loro,"5' 7""",135 lbs.,--,Orthodox,"Jun 23, 1981",Record: 18-8-1,1.97,32%,3.86,61%,2.27,28%,88%,1.1
4,Herdem Alacabek,,"6' 2""",205 lbs.,"75""",Orthodox,"Jun 07, 1991",Record: 5-1-0,3.84,65%,6.66,23%,2.06,50%,42%,0.0


#### HEIGHT to INCHES
* First extract numerical values from the heights column
* Convert the height from string type to float type
* Multiply feet by 12 to get it to inches add the "feet" with inches to get total height in inches

In [7]:
#extract numerical values from the heights column
fighters_df[['feet','inches']] = fighters_df['height'].str.extract("(\d+)' (\d+)")

#convert it into float, because we have NULL values
fighters_df[['feet','inches']] = fighters_df[['feet','inches']].astype('float').copy()

#multiply feet by 12 and add the inches to replace old heigh column
fighters_df['height'] = (fighters_df['feet']*12) + fighters_df['inches']

#drop the feet and inches because now we have height
fighters_df.drop(labels = ['feet','inches'],axis =1, inplace =True)

#### EXTRACT  Numerical Values From Column
* Create function to convert character to number
* Turn weight,reach,StrAcc,StrDef,TDacc,TDdef into float

In [8]:
def char2num(df,column):
    converted = df[column].str.extract("(\d+)").astype('float').copy()
    return converted

In [9]:
#using char2num turn weight,reach,StrAcc,StrDef,TDacc,TDdef into float
fighters_df['weight'] = char2num(fighters_df,'weight')
fighters_df['reach'] = char2num(fighters_df,'reach')
fighters_df['StrAcc'] = char2num(fighters_df,'StrAcc')
fighters_df['StrDef'] = char2num(fighters_df,'StrDef')
fighters_df['TDacc'] = char2num(fighters_df,'TDacc')
fighters_df['TDdef'] = char2num(fighters_df,'TDdef')

#### STRING to DATE
* Convert date_birth from String to DateTime Object

In [10]:
#coerce the errors missing values = NaT
fighters_df['date_birth'] = pd.to_datetime(fighters_df['date_birth'],errors= 'coerce')

#### EXTRACT WIN LOSE DRAW FROM RECORD

In [11]:
fighters_df['record'].head()

0      Record: 6-3-0
1      Record: 0-1-0
2    Record: 42-21-0
3     Record: 18-8-1
4      Record: 5-1-0
Name: record, dtype: object

In [12]:
#extract the strings and then convert into int type
fighters_df[['Win','Lose','Draw']] = fighters_df['record'].str.extract("(\d+)-(\d+)-(\d+)").astype('int')

#drop the records column
fighters_df.drop('record',axis=1,inplace = True)

### HANDLING MISSING VALUES
> Drop records that have more than 3 missing columns    
> Impute missing heights with the average height according to fighter's weight

In [13]:
fighters_df.isnull().sum()

name             0
nick_name        0
height         261
weight          75
reach         1927
stance           0
date_birth     742
SLpm             0
StrAcc           0
SApm             0
StrDef           0
TDavg            0
TDacc            0
TDdef            0
SubAvg           0
Win              0
Lose             0
Draw             0
dtype: int64

#### More than 3 Missing Values
* Find the records where there are more than 3 missing values per fighter
* Drop those records

In [14]:
#drop all records where there are more than 3 missing values
#because fighters we more than 3 missing values seem to be outdated
fighters_df.dropna(axis =0,thresh=fighters_df.shape[1]- 2, inplace = True)

In [15]:
fighters_df.isnull().sum()

name             0
nick_name        0
height          53
weight           1
reach         1714
stance           0
date_birth     530
SLpm             0
StrAcc           0
SApm             0
StrDef           0
TDavg            0
TDacc            0
TDdef            0
SubAvg           0
Win              0
Lose             0
Draw             0
dtype: int64

#### FILL IN HEIGHT
* Map the average heights to the missing heights accoring to the fighters weight
* GET values where all values of height are null
* Group the fighters_df by the weight to get the average height by weight
* Map the missing values of height with the average height of its respective weight
* Fill in all NULL height values with its respective weight averages

In [16]:
#values for of all columns in height that are null
null_height = fighters_df[fighters_df['height'].isnull()]

#get the average height by weight
average_by_weight = fighters_df.groupby('weight').mean()
average_by_weight.head()

,height,reach,StrAcc,StrDef,TDacc,TDdef,Win,Lose,Draw
weight,,,,,,,,,
105.0,62.000000,NaN,34.000000,74.500000,50.000000,33.000000,10.500000,7.500000,0.000000
113.0,63.000000,NaN,21.000000,78.500000,0.000000,50.000000,20.000000,7.000000,0.500000
115.0,63.609756,63.633333,40.939759,48.228916,31.048193,49.156627,8.626506,3.891566,0.048193
125.0,65.714286,66.708029,37.317460,50.158730,30.142857,45.052910,11.153439,4.423280,0.158730
130.0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,3.500000,3.500000,0.000000


In [17]:
#map each missing value of heights with 
missing_height_weight_mapped = null_height['weight'].map(average_by_weight['height']).copy()
missing_height_weight_mapped.head()

41     72.197590
52     72.197590
59     71.104247
318    63.609756
418    68.456522
Name: weight, dtype: float64

In [18]:
#fill in the na values 
fighters_df['height'].fillna(missing_height_weight_mapped,inplace = True)

In [19]:
print("Now there are only {} missing height from the original".format(fighters_df.isnull().sum().loc['height']))

Now there are only 2 missing height from the original


In [20]:
fighters_df[fighters_df['height'].isnull()]

,name,nick_name,height,weight,reach,stance,date_birth,SLpm,StrAcc,SApm,StrDef,TDavg,TDacc,TDdef,SubAvg,Win,Lose,Draw
2712,Josh Epps,,NaN,130.0,NaN,,1986-08-23,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,6,7,0
2904,JW Wright,,NaN,130.0,NaN,,1976-01-12,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,1,0,0


#### FILLING IN REACH 

In [21]:
fighters_df.isnull().sum()

name             0
nick_name        0
height           2
weight           1
reach         1714
stance           0
date_birth     530
SLpm             0
StrAcc           0
SApm             0
StrDef           0
TDavg            0
TDacc            0
TDdef            0
SubAvg           0
Win              0
Lose             0
Draw             0
dtype: int64

In [22]:
fighters_df[fighters_df['reach'].isnull()]

,name,nick_name,height,weight,reach,stance,date_birth,SLpm,StrAcc,SApm,StrDef,TDavg,TDacc,TDdef,SubAvg,Win,Lose,Draw
1,Scott Fiedler,,76.000000,235.0,NaN,Orthodox,NaT,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0,1,0
3,Marcos Galvao,Loro,67.000000,135.0,NaN,Orthodox,1981-06-23,1.97,32.0,3.86,61.0,2.27,28.0,88.0,1.1,18,8,1
5,Jorge Interiano,,73.000000,205.0,NaN,,NaT,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,4,3,0
7,Steve Jennum,,70.000000,215.0,NaN,Orthodox,NaT,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,2,3,0
8,Kyle Jensen,,69.000000,170.0,NaN,Orthodox,1979-07-20,0.00,0.0,0.92,60.0,6.92,100.0,0.0,6.9,4,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3545,Dave Zitanick,,71.104247,170.0,NaN,,1980-03-05,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,5,7,0
3546,Igor Zinoviev,Houdini,73.000000,199.0,NaN,Orthodox,NaT,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,4,1,2
3549,Errol Zimmerman,The Bonecrusher,75.000000,185.0,NaN,Orthodox,1986-04-20,2.95,42.0,0.00,0.0,0.00,0.0,0.0,0.0,0,1,0
3550,James Zikic,The Messenger,74.000000,205.0,NaN,Orthodox,1977-03-01,1.47,35.0,1.60,44.0,0.50,25.0,74.0,0.5,21,10,2


In [23]:
#create fighters csv in current directory
fighters_df.to_csv('fighters.csv',index= False)